# Test: generate_ontology_tables() Function

This notebook validates the `generate_ontology_tables()` function that creates
ontology tables from a clade folder's db.sqlite file.

**Author:** Jose P. Faria (jplfaria@gmail.com)  
**Date:** February 2026

## What this tests:
1. Loading genome features from SQLite
2. Extracting ontology terms (GO, EC, KEGG, COG, PFAM, SO, seed.role)
3. Enriching terms from local parquet files
4. Extracting relationships (GO is_a, seed.role → seed.reaction)
5. Saving output tables (ontology_terms.tsv, ontology_definition.tsv, ontology_relationships.tsv)

In [1]:
import sys
import os
import sqlite3
import pandas as pd
from pathlib import Path

# Add the KBDatalakeApps lib to path
sys.path.insert(0, '/home/jplfaria/repos/KBDatalakeApps/lib/KBDatalakeApps')

print("Python version:", sys.version)
print("Pandas version:", pd.__version__)

Python version: 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
Pandas version: 2.2.3


## 1. Check Available Test Data

In [2]:
# Local parquet files location
PARQUET_PATH = Path('/home/jplfaria/CDM_jose/ontology_data_utils/run_20250819_020438/parquet_files')

# Test database (we'll copy this to a mock clade folder)
TEST_DB = Path('/home/jplfaria/CDM_jose/ontology_data_utils/berdl_tables.db')

# Check files exist
print("=" * 60)
print("Checking test data availability...")
print("=" * 60)

required_files = [
    PARQUET_PATH / 'statements.parquet',
    PARQUET_PATH / 'kegg_ko_definitions.parquet',
    PARQUET_PATH / 'cog_definitions.parquet',
    PARQUET_PATH / 'seed.json',  # Required for RAST → seed.role mapping
    PARQUET_PATH / 'kegg_ko_ec_mapping.tsv',  # Required for KEGG KO → EC mapping
]

for f in required_files:
    exists = f.exists()
    size = f.stat().st_size / 1024 / 1024 if exists else 0
    status = f"OK ({size:.2f} MB)" if exists else "MISSING"
    print(f"  {f.name}: {status}")

print(f"\n  Test database: {'OK' if TEST_DB.exists() else 'MISSING'}")

Checking test data availability...
  statements.parquet: OK (313.44 MB)
  kegg_ko_definitions.parquet: OK (0.89 MB)
  cog_definitions.parquet: OK (0.19 MB)
  seed.json: OK (14.55 MB)
  kegg_ko_ec_mapping.tsv: OK (0.21 MB)

  Test database: OK


In [3]:
# Check what tables are in the test database
if TEST_DB.exists():
    conn = sqlite3.connect(str(TEST_DB))
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
    print("Tables in test database:")
    for t in tables['name']:
        count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM {t}", conn)['cnt'][0]
        print(f"  {t}: {count:,} rows")
    conn.close()

Tables in test database:
  phenotype_module: 409 rows
  genome: 42 rows
  genome_ani: 7 rows
  genome_features: 4,617 rows
  pan_genome_features: 165,496 rows
  ontology_definition: 8 rows
  ontology_relationships: 7,483 rows
  ontology_terms: 15,719 rows


## 2. Create Mock Clade Folder for Testing

In [4]:
import shutil

# Create a mock clade folder structure
MOCK_CLADE_FOLDER = Path('/home/jplfaria/CDM_jose/ontology_data_utils/test_clade_s__Escherichia_coli')

# Clean up if exists
if MOCK_CLADE_FOLDER.exists():
    shutil.rmtree(MOCK_CLADE_FOLDER)

# Create folder structure
MOCK_CLADE_FOLDER.mkdir(parents=True, exist_ok=True)

# Copy the test database as db.sqlite
shutil.copy(TEST_DB, MOCK_CLADE_FOLDER / 'db.sqlite')

print(f"Created mock clade folder: {MOCK_CLADE_FOLDER}")
print(f"Contents:")
for f in MOCK_CLADE_FOLDER.iterdir():
    print(f"  {f.name}")

Created mock clade folder: /home/jplfaria/CDM_jose/ontology_data_utils/test_clade_s__Escherichia_coli
Contents:
  db.sqlite


## 3. Import and Run generate_ontology_tables()

In [5]:
# Define RASTSeedMapper class for RAST → seed.role mapping
# This is copied from KBDatalakeUtils.py for standalone testing

import json

class RASTSeedMapper:
    """Maps RAST annotations to SEED role ontology identifiers."""
    
    def __init__(self, seed_ontology_path: str):
        self.seed_mapping = {}
        self.multi_func_separators = [' / ', ' @ ', '; ']
        self._load_seed_ontology(seed_ontology_path)
    
    def _load_seed_ontology(self, path: str) -> None:
        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(f"Ontology file not found: {path}")
        
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        graphs = data.get("graphs", [])
        if not graphs:
            print("Warning: No graphs found in ontology file")
            return
            
        nodes = graphs[0].get("nodes", [])
        
        for node in nodes:
            label = node.get("lbl")
            node_id = node.get("id")
            
            if not label or not node_id:
                continue
                
            seed_role_id = self._parse_seed_role_id(node_id)
            if seed_role_id:
                self.seed_mapping[label] = seed_role_id
                
        print(f"    Loaded {len(self.seed_mapping)} SEED role mappings")
        
    def _parse_seed_role_id(self, raw_id: str) -> str:
        if not raw_id:
            return None
        if "Role=" in raw_id:
            try:
                role_number = raw_id.split("Role=")[-1]
                return f"seed.role:{role_number}"
            except IndexError:
                return None
        if raw_id.startswith("seed.role:"):
            return raw_id
        if '_' in raw_id and 'seed.role_' in raw_id:
            ontology_part = raw_id.split('/')[-1]
            return ontology_part.replace("_", ":", 1)
        return None
    
    def split_multi_function(self, annotation: str) -> list:
        if not annotation:
            return []
        parts = [annotation]
        for separator in self.multi_func_separators:
            new_parts = []
            for part in parts:
                split_parts = part.split(separator)
                new_parts.extend(p.strip() for p in split_parts if p.strip())
            parts = new_parts
        return parts
    
    def map_annotation(self, annotation: str) -> str:
        if not annotation:
            return None
        if annotation in self.seed_mapping:
            return self.seed_mapping[annotation]
        parts = self.split_multi_function(annotation)
        if len(parts) > 1:
            for part in parts:
                if part in self.seed_mapping:
                    return self.seed_mapping[part]
        return None
    
    def map_all_annotations(self, annotation: str) -> list:
        if not annotation:
            return []
        results = []
        if annotation in self.seed_mapping:
            results.append((annotation, self.seed_mapping[annotation]))
        parts = self.split_multi_function(annotation)
        for part in parts:
            if part in self.seed_mapping and part != annotation:
                results.append((part, self.seed_mapping[part]))
        return results

print("RASTSeedMapper class defined successfully!")

RASTSeedMapper class defined successfully!


In [6]:
# Full generate_ontology_tables function with RAST mapping, EC column, seed.reaction
# Copied from KBDatalakeUtils.py for standalone testing
# Updated: Added GO → EC extraction from oio:hasDbXref

import sqlite3
import re
import time
from pathlib import Path
import pyarrow.parquet as pq

def generate_ontology_tables(
    clade_folder: str,
    reference_data_path: str = "/data/reference_data",
    genome_features_table: str = "genome_features",
    output_folder_name: str = "ontology_data"
) -> bool:
    """Generate ontology tables for a clade folder with RAST→seed.role mapping and EC column."""
    
    clade_path = Path(clade_folder)
    db_path = clade_path / "db.sqlite"
    output_path = clade_path / output_folder_name
    ref_path = Path(reference_data_path)

    if not db_path.exists():
        print(f"Warning: db.sqlite not found in {clade_folder}")
        return False

    print(f"\n{'='*70}")
    print(f"Generating ontology tables for: {clade_folder}")
    print(f"{'='*70}")

    try:
        # STEP 1: Load genome features
        print(f"\n1. Loading genome features from {db_path}...")
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{genome_features_table}'")
        if not cursor.fetchone():
            print(f"   Warning: Table '{genome_features_table}' not found")
            conn.close()
            return False

        genome_df = pd.read_sql_query(f"SELECT * FROM {genome_features_table}", conn)
        conn.close()
        print(f"   Loaded {len(genome_df)} features")

        # STEP 2: Initialize RASTSeedMapper
        print("\n2. Loading RAST → seed.role mapper...")
        seed_json_path = ref_path / "seed.json"
        mapper = None
        if seed_json_path.exists():
            mapper = RASTSeedMapper(str(seed_json_path))
        else:
            print(f"   Warning: seed.json not found")

        # STEP 3: Extract ontology terms
        print("\n3. Extracting ontology terms...")
        terms_by_type = {'GO': set(), 'EC': set(), 'KEGG': set(), 'COG': set(), 'PFAM': set(), 'SO': set(), 'seed.role': set()}
        patterns = {
            'GO': re.compile(r'GO:\d+'), 'EC': re.compile(r'EC:[\d\.-]+'),
            'KEGG': re.compile(r'(?:KEGG:)?K\d{5}'), 'COG': re.compile(r'COG:(?:COG\d+|[A-Z])'),
            'PFAM': re.compile(r'(?:PFAM:)?PF\d+(?:\.\d+)?'), 'SO': re.compile(r'SO:\d+'),
            'seed.role': re.compile(r'seed\.role:\d+'),
        }
        ec_in_rast_pattern = re.compile(r'\(EC[:\s]*([\d\.-]+)\)')
        rast_functions = set()
        seed_role_to_label = {}
        
        rast_col = None
        for col in ['rast_function', 'rast_functions', 'functions', 'Annotation:SSO']:
            if col in genome_df.columns:
                rast_col = col
                break
        if rast_col:
            print(f"   Using RAST function column: {rast_col}")
        
        for col in genome_df.columns:
            for _, row in genome_df.iterrows():
                value = str(row.get(col, ''))
                if not value or value == 'nan':
                    continue
                for ont_type, pattern in patterns.items():
                    matches = pattern.findall(value)
                    for match in matches:
                        if ont_type == 'KEGG' and not match.startswith('KEGG:'):
                            match = f'KEGG:{match}'
                        elif ont_type == 'PFAM' and not match.startswith('PFAM:'):
                            match = f'PFAM:{match}'
                        terms_by_type[ont_type].add(match)
                if col == rast_col:
                    ec_matches = ec_in_rast_pattern.findall(value)
                    for ec_num in ec_matches:
                        terms_by_type['EC'].add(f'EC:{ec_num}')
                    if value and value != 'nan':
                        for separator in [' / ', ' @ ', '; ']:
                            if separator in value:
                                for part in value.split(separator):
                                    part = part.strip()
                                    if part:
                                        rast_functions.add(part)
                        if not any(sep in value for sep in [' / ', ' @ ', '; ']):
                            rast_functions.add(value)

        # STEP 4: Map RAST functions to seed.role IDs
        if mapper and rast_functions:
            print(f"\n4. Mapping {len(rast_functions)} RAST functions to seed.role IDs...")
            for rast_func in rast_functions:
                for matched_part, seed_id in mapper.map_all_annotations(rast_func):
                    if seed_id:
                        terms_by_type['seed.role'].add(seed_id)
                        seed_role_to_label[seed_id] = matched_part
            print(f"   Mapped to {len(terms_by_type['seed.role'])} unique seed.role IDs")
        else:
            print("\n4. Skipping RAST → seed.role mapping")

        total_terms = sum(len(terms) for terms in terms_by_type.values())
        print(f"\n   Total unique terms: {total_terms}")
        for ont_type, terms in sorted(terms_by_type.items()):
            if terms:
                print(f"     {ont_type}: {len(terms)}")

        if total_terms == 0:
            os.makedirs(output_path, exist_ok=True)
            pd.DataFrame(columns=['ontology_prefix', 'identifier', 'label', 'definition', 'ec']).to_csv(output_path / 'ontology_terms.tsv', sep='\t', index=False)
            pd.DataFrame(columns=['ontology_prefix', 'definition']).to_csv(output_path / 'ontology_definition.tsv', sep='\t', index=False)
            pd.DataFrame(columns=['subject', 'predicate', 'object']).to_csv(output_path / 'ontology_relationships.tsv', sep='\t', index=False)
            return True

        # STEP 5: Enrich terms from parquet files
        print("\n5. Enriching terms from local parquet files...")
        enriched_terms = []
        statements_df = None
        statements_path = ref_path / "statements.parquet"
        kegg_path = ref_path / "kegg_ko_definitions.parquet"
        cog_path = ref_path / "cog_definitions.parquet"
        
        berdl_terms = list(terms_by_type['GO'] | terms_by_type['EC'] | terms_by_type['SO'] | terms_by_type['PFAM'] | terms_by_type['seed.role'])
        
        if berdl_terms and statements_path.exists():
            print(f"   Loading statements.parquet...")
            statements_df = pq.read_table(statements_path).to_pandas()
            mask = statements_df['subject'].isin(berdl_terms) & statements_df['predicate'].isin(['rdfs:label', 'IAO:0000115'])
            filtered = statements_df[mask]
            term_info = {}
            for _, row in filtered.iterrows():
                subj, pred = row['subject'], row['predicate']
                val = row['value'] if 'value' in row else ''
                if subj not in term_info:
                    term_info[subj] = {'label': '', 'definition': ''}
                if pred == 'rdfs:label':
                    term_info[subj]['label'] = val
                elif pred == 'IAO:0000115':
                    term_info[subj]['definition'] = val
            for term_id in berdl_terms:
                prefix = term_id.split(':')[0]
                info = term_info.get(term_id, {'label': '', 'definition': ''})
                label = info['label']
                if prefix == 'seed.role' and not label and term_id in seed_role_to_label:
                    label = seed_role_to_label[term_id]
                enriched_terms.append({'ontology_prefix': prefix, 'identifier': term_id, 'label': label, 'definition': info['definition']})
            print(f"   Enriched {len([t for t in enriched_terms if t['label']])} terms with labels")
        
        if terms_by_type['KEGG'] and kegg_path.exists():
            kegg_df = pq.read_table(kegg_path).to_pandas()
            ko_lookup = dict(zip(kegg_df['ko_id'], kegg_df['definition']))
            for ko_id in terms_by_type['KEGG']:
                k_num = ko_id.replace('KEGG:', '')
                definition = ko_lookup.get(k_num, '')
                label = re.sub(r'\s*\[EC:[^\]]+\]', '', definition).strip() if definition else ''
                enriched_terms.append({'ontology_prefix': 'KEGG', 'identifier': ko_id, 'label': label, 'definition': definition})
        
        if terms_by_type['COG'] and cog_path.exists():
            cog_df = pq.read_table(cog_path).to_pandas()
            cog_lookup = {row['cog_id']: row for _, row in cog_df.iterrows()}
            for cog_id in terms_by_type['COG']:
                raw_id = cog_id.replace('COG:', '')
                info = cog_lookup.get(raw_id, {})
                enriched_terms.append({'ontology_prefix': 'COG', 'identifier': cog_id, 'label': info.get('name', '') if isinstance(info, dict) else '', 'definition': info.get('pathway', '') if isinstance(info, dict) else ''})

        # STEP 6: Extract relationships
        print("\n6. Extracting ontology relationships...")
        relationships = []
        all_term_ids = set()
        for terms in terms_by_type.values():
            all_term_ids.update(terms)
        seed_reaction_terms = set()
        
        if statements_df is not None:
            relevant_predicates = {'rdfs:subClassOf', '<https://modelseed.org/ontology/enables_reaction>'}
            mask = statements_df['subject'].isin(all_term_ids) & statements_df['predicate'].isin(relevant_predicates)
            rel_df = statements_df[mask]
            predicate_labels = {'rdfs:subClassOf': 'is_a', '<https://modelseed.org/ontology/enables_reaction>': 'enables_reaction'}
            for _, row in rel_df.iterrows():
                subj, pred, obj = row['subject'], row['predicate'], row['object']
                if subj == obj or str(obj).startswith('_:'):
                    continue
                if pred == 'rdfs:subClassOf' and (subj.startswith('EC:') or subj.startswith('SO:')):
                    continue
                if str(obj).startswith('seed.reaction:'):
                    seed_reaction_terms.add(obj)
                relationships.append({'subject': subj, 'predicate': predicate_labels.get(pred, pred), 'object': obj})
            print(f"   Found {len(relationships)} relationships, {len(seed_reaction_terms)} seed.reaction terms")
            
            if seed_reaction_terms:
                mask = statements_df['subject'].isin(seed_reaction_terms) & statements_df['predicate'].isin(['rdfs:label', 'IAO:0000115'])
                rxn_info = {}
                for _, row in statements_df[mask].iterrows():
                    subj, pred = row['subject'], row['predicate']
                    val = row['value'] if 'value' in row else ''
                    if subj not in rxn_info:
                        rxn_info[subj] = {'label': '', 'definition': ''}
                    if pred == 'rdfs:label':
                        rxn_info[subj]['label'] = val
                    elif pred == 'IAO:0000115':
                        rxn_info[subj]['definition'] = val
                for rxn_id in seed_reaction_terms:
                    info = rxn_info.get(rxn_id, {'label': '', 'definition': ''})
                    enriched_terms.append({'ontology_prefix': 'seed.reaction', 'identifier': rxn_id, 'label': info['label'], 'definition': info['definition']})

        # STEP 7: Add EC column (including GO → EC from hasDbXref)
        print("\n7. Adding EC column to ontology terms...")
        
        # Load KEGG KO → EC mapping
        kegg_ec_mapping_path = ref_path / "kegg_ko_ec_mapping.tsv"
        ko_to_ec = {}
        if kegg_ec_mapping_path.exists():
            with open(kegg_ec_mapping_path, 'r') as f:
                for line in f:
                    line = line.strip()
                    if '\t' in line:
                        ec_raw, ko_raw = line.split('\t')
                        ec_id = ec_raw.replace('ec:', 'EC:')
                        ko_id = ko_raw.replace('ko:', 'KEGG:')
                        if ko_id not in ko_to_ec:
                            ko_to_ec[ko_id] = []
                        ko_to_ec[ko_id].append(ec_id)
            print(f"   Loaded {len(ko_to_ec)} KEGG KO -> EC mappings")
        
        # Extract GO → EC mapping from statements.parquet (oio:hasDbXref to EC:)
        go_to_ec = {}
        if statements_df is not None:
            print(f"   Extracting GO -> EC mappings from statements.parquet...")
            go_ec_mask = (
                statements_df['subject'].str.startswith('GO:', na=False) &
                (statements_df['predicate'] == 'oio:hasDbXref')
            )
            go_dbxref_df = statements_df[go_ec_mask]
            
            ec_pattern = re.compile(r'EC:[\d\.\-]+')
            for _, row in go_dbxref_df.iterrows():
                go_id = row['subject']
                # Check both object and value columns for EC reference
                obj_val = str(row.get('object', '')) + ' ' + str(row.get('value', ''))
                ec_matches = ec_pattern.findall(obj_val)
                if ec_matches:
                    if go_id not in go_to_ec:
                        go_to_ec[go_id] = []
                    go_to_ec[go_id].extend(ec_matches)
            
            # Deduplicate EC values per GO term
            for go_id in go_to_ec:
                go_to_ec[go_id] = list(set(go_to_ec[go_id]))
            
            print(f"   Found {len(go_to_ec)} GO terms with EC cross-references")
        
        ec_label_pattern = re.compile(r'\(EC\s*([\d\.-]+)\)')
        tc_label_pattern = re.compile(r'\(TC\s*([\d\.\w]+)\)')
        kegg_ec_count, go_ec_count, seed_ec_count, seed_tc_count, ec_copy_count = 0, 0, 0, 0, 0
        
        for term in enriched_terms:
            ec_values = []
            prefix, identifier, label = term['ontology_prefix'], term['identifier'], term.get('label', '')
            
            if prefix == 'KEGG' and identifier in ko_to_ec:
                ec_values.extend(ko_to_ec[identifier])
                kegg_ec_count += 1
            elif prefix == 'GO' and identifier in go_to_ec:
                ec_values.extend(go_to_ec[identifier])
                go_ec_count += 1
            elif prefix == 'seed.role' and label:
                ec_matches = ec_label_pattern.findall(label)
                if ec_matches:
                    ec_values.extend(['EC:' + m for m in ec_matches])
                    seed_ec_count += 1
                tc_matches = tc_label_pattern.findall(label)
                if tc_matches:
                    ec_values.extend(['TC:' + m for m in tc_matches])
                    seed_tc_count += 1
            elif prefix == 'EC':
                ec_values.append(identifier)
                ec_copy_count += 1
            term['ec'] = '|'.join(ec_values) if ec_values else ''
        
        print(f"   KEGG with EC: {kegg_ec_count}, GO with EC: {go_ec_count}, seed.role EC: {seed_ec_count}, TC: {seed_tc_count}, EC copied: {ec_copy_count}")
        print(f"   Total terms with ec: {sum(1 for t in enriched_terms if t.get('ec'))}")

        # STEP 8: Create ontology definitions
        print("\n8. Creating ontology definitions...")
        ontology_definitions = {
            'GO': 'Gene Ontology', 'EC': 'Enzyme Commission numbers', 'SO': 'Sequence Ontology',
            'PFAM': 'Protein Families database', 'KEGG': 'KEGG Orthologs', 'COG': 'Clusters of Orthologous Groups',
            'seed.role': 'SEED Role Ontology', 'seed.reaction': 'SEED Reaction Ontology',
        }
        present_prefixes = set(t['ontology_prefix'] for t in enriched_terms)
        definition_rows = [{'ontology_prefix': p, 'definition': d} for p, d in ontology_definitions.items() if p in present_prefixes]

        # STEP 9: Save output
        print(f"\n9. Saving output to {output_path}...")
        os.makedirs(output_path, exist_ok=True)
        terms_df = pd.DataFrame(enriched_terms).drop_duplicates(subset=['identifier'])
        terms_df = terms_df.sort_values(['ontology_prefix', 'identifier']).reset_index(drop=True)
        terms_df.to_csv(output_path / 'ontology_terms.tsv', sep='\t', index=False)
        print(f"   Saved {len(terms_df)} terms to ontology_terms.tsv")
        print(f"   Columns: {list(terms_df.columns)}")
        
        pd.DataFrame(definition_rows).to_csv(output_path / 'ontology_definition.tsv', sep='\t', index=False)
        rels_df = pd.DataFrame(relationships)
        if not rels_df.empty:
            rels_df = rels_df.drop_duplicates()
        rels_df.to_csv(output_path / 'ontology_relationships.tsv', sep='\t', index=False)
        print(f"   Saved {len(rels_df)} relationships")
        
        print(f"\n{'='*70}")
        print("Ontology table generation complete!")
        print(f"{'='*70}")
        return True

    except Exception as e:
        import traceback
        print(f"\nError: {e}")
        print(traceback.format_exc())
        return False

print("generate_ontology_tables function defined successfully!")

generate_ontology_tables function defined successfully!


In [7]:
# Run the function against our mock clade folder
result = generate_ontology_tables(
    clade_folder=str(MOCK_CLADE_FOLDER),
    reference_data_path=str(PARQUET_PATH),
    genome_features_table="genome_features",
    output_folder_name="ontology_data"
)

print(f"\nResult: {'SUCCESS' if result else 'FAILED'}")


Generating ontology tables for: /home/jplfaria/CDM_jose/ontology_data_utils/test_clade_s__Escherichia_coli

1. Loading genome features from /home/jplfaria/CDM_jose/ontology_data_utils/test_clade_s__Escherichia_coli/db.sqlite...
   Loaded 4617 features

2. Loading RAST → seed.role mapper...
    Loaded 46232 SEED role mappings

3. Extracting ontology terms...
   Using RAST function column: rast_function

4. Mapping 3952 RAST functions to seed.role IDs...
   Mapped to 3951 unique seed.role IDs

   Total unique terms: 12009
     COG: 1670
     EC: 1268
     GO: 3544
     KEGG: 1485
     PFAM: 90
     SO: 1
     seed.role: 3951

5. Enriching terms from local parquet files...
   Loading statements.parquet...
   Enriched 8612 terms with labels

6. Extracting ontology relationships...
   Found 7483 relationships, 1443 seed.reaction terms

7. Adding EC column to ontology terms...
   Loaded 8981 KEGG KO -> EC mappings
   Extracting GO -> EC mappings from statements.parquet...
   Found 4561 GO t

## 4. Verify Output Files

In [8]:
# Check output folder
output_folder = MOCK_CLADE_FOLDER / 'ontology_data'

print("Output folder contents:")
if output_folder.exists():
    for f in output_folder.iterdir():
        size = f.stat().st_size / 1024
        print(f"  {f.name}: {size:.1f} KB")
else:
    print("  Output folder not created!")

Output folder contents:
  ontology_terms.tsv: 1455.7 KB
  ontology_definition.tsv: 0.2 KB
  ontology_relationships.tsv: 274.3 KB


In [9]:
# Read and display ontology_terms.tsv
terms_file = output_folder / 'ontology_terms.tsv'
if terms_file.exists():
    terms_df = pd.read_csv(terms_file, sep='\t')
    print(f"ontology_terms.tsv: {len(terms_df)} rows")
    print(f"\nColumns: {list(terms_df.columns)}")
    print(f"\nBy ontology_prefix:")
    print(terms_df['ontology_prefix'].value_counts())
    print(f"\nSample rows:")
    display(terms_df.head(10))

ontology_terms.tsv: 13452 rows

Columns: ['ontology_prefix', 'identifier', 'label', 'definition', 'ec']

By ontology_prefix:
ontology_prefix
seed.role        3951
GO               3544
COG              1670
KEGG             1485
seed.reaction    1443
EC               1268
PFAM               90
SO                  1
Name: count, dtype: int64

Sample rows:


,ontology_prefix,identifier,label,definition,ec
0,COG,COG:A,NaN,NaN,NaN
1,COG,COG:B,NaN,NaN,NaN
2,COG,COG:C,NaN,NaN,NaN
3,COG,COG:COG0001,NaN,NaN,NaN
4,COG,COG:COG0002,NaN,NaN,NaN
5,COG,COG:COG0004,NaN,NaN,NaN
6,COG,COG:COG0006,NaN,NaN,NaN
7,COG,COG:COG0008,NaN,NaN,NaN
8,COG,COG:COG0009,NaN,NaN,NaN
9,COG,COG:COG0010,NaN,NaN,NaN


In [10]:
# Read and display ontology_definition.tsv
defs_file = output_folder / 'ontology_definition.tsv'
if defs_file.exists():
    defs_df = pd.read_csv(defs_file, sep='\t')
    print(f"ontology_definition.tsv: {len(defs_df)} rows")
    display(defs_df)

ontology_definition.tsv: 8 rows


,ontology_prefix,definition
0,GO,Gene Ontology
1,EC,Enzyme Commission numbers
2,SO,Sequence Ontology
3,PFAM,Protein Families database
4,KEGG,KEGG Orthologs
5,COG,Clusters of Orthologous Groups
6,seed.role,SEED Role Ontology
7,seed.reaction,SEED Reaction Ontology


In [11]:
# Read and display ontology_relationships.tsv
rels_file = output_folder / 'ontology_relationships.tsv'
if rels_file.exists():
    rels_df = pd.read_csv(rels_file, sep='\t')
    print(f"ontology_relationships.tsv: {len(rels_df)} rows")
    print(f"\nColumns: {list(rels_df.columns)}")
    
    if len(rels_df) > 0:
        print(f"\nBy predicate:")
        print(rels_df['predicate'].value_counts())
        
        print(f"\nSample relationships:")
        display(rels_df.head(10))

ontology_relationships.tsv: 7483 rows

Columns: ['subject', 'predicate', 'object']

By predicate:
predicate
is_a                5354
enables_reaction    2129
Name: count, dtype: int64

Sample relationships:


,subject,predicate,object
0,GO:0000006,is_a,GO:0005385
1,GO:0000014,is_a,GO:0016788
2,GO:0000014,is_a,GO:0004520
3,GO:0000015,is_a,GO:1902494
4,GO:0000018,is_a,GO:0051052
5,GO:0000023,is_a,GO:0005984
6,GO:0000025,is_a,GO:0046352
7,GO:0000025,is_a,GO:0000023
8,GO:0000027,is_a,GO:0022618
9,GO:0000028,is_a,GO:0022618


In [12]:
# Check specifically for seed.role, seed.reaction, and EC column (new features)
terms_file = output_folder / 'ontology_terms.tsv'
rels_file = output_folder / 'ontology_relationships.tsv'

if terms_file.exists():
    terms_df = pd.read_csv(terms_file, sep='\t')
    
    print("=" * 60)
    print("VERIFICATION: New Features (including GO → EC)")
    print("=" * 60)
    
    # Check columns include 'ec'
    print(f"\nColumns: {list(terms_df.columns)}")
    
    # Check for EC column content
    if 'ec' in terms_df.columns:
        ec_filled = terms_df[terms_df['ec'].notna() & (terms_df['ec'] != '')]
        print(f"\nEC column: {len(ec_filled)} terms have EC/TC values")
        
        # By prefix
        print("\nEC column by ontology prefix:")
        for prefix in terms_df['ontology_prefix'].unique():
            mask = (terms_df['ontology_prefix'] == prefix) & terms_df['ec'].notna() & (terms_df['ec'] != '')
            count = mask.sum()
            if count > 0:
                print(f"  {prefix}: {count}")
        
        # Show GO terms with EC (new feature!)
        print("\n" + "=" * 60)
        print("GO terms with EC cross-references (from oio:hasDbXref):")
        print("=" * 60)
        go_with_ec = terms_df[(terms_df['ontology_prefix'] == 'GO') & terms_df['ec'].notna() & (terms_df['ec'] != '')]
        print(f"Total GO terms with EC: {len(go_with_ec)}")
        if len(go_with_ec) > 0:
            print("\nSample GO → EC mappings:")
            display(go_with_ec[['identifier', 'label', 'ec']].head(15))
    
    # Check for seed.role
    seed_roles = terms_df[terms_df['ontology_prefix'] == 'seed.role']
    print(f"\nseed.role terms: {len(seed_roles)}")
    
    # Check for seed.reaction
    seed_reactions = terms_df[terms_df['ontology_prefix'] == 'seed.reaction']
    print(f"seed.reaction terms: {len(seed_reactions)}")

if rels_file.exists():
    rels_df = pd.read_csv(rels_file, sep='\t')
    enables_reaction = rels_df[rels_df['predicate'] == 'enables_reaction']
    print(f"enables_reaction relationships: {len(enables_reaction)}")

VERIFICATION: New Features (including GO → EC)

Columns: ['ontology_prefix', 'identifier', 'label', 'definition', 'ec']

EC column: 4204 terms have EC/TC values

EC column by ontology prefix:
  EC: 1268
  GO: 1100
  KEGG: 613
  seed.role: 1223

GO terms with EC cross-references (from oio:hasDbXref):
Total GO terms with EC: 1100

Sample GO → EC mappings:


,identifier,label,ec
2947,GO:0000034,adenine deaminase activity,EC:3.5.4.2
2970,GO:0000215,tRNA 2'-phosphotransferase activity,EC:2.7.1.160
2979,GO:0000309,nicotinamide-nucleotide adenylyltransferase ac...,EC:2.7.7.1
2980,GO:0000310,xanthine phosphoribosyltransferase activity,EC:2.4.2.22
2990,GO:0000701,purine-specific mismatch base pair DNA N-glyco...,EC:3.2.2.31
2998,GO:0000810,diacylglycerol diphosphate phosphatase activity,EC:3.6.1.75
3001,GO:0000906,"6,7-dimethyl-8-ribityllumazine synthase activity",EC:2.5.1.78
3002,GO:0000908,taurine dioxygenase activity,EC:1.14.11.17
3064,GO:0002952,"(4S)-4-hydroxy-5-phosphonooxypentane-2,3-dione...",EC:5.3.1.32
3065,GO:0002953,5'-deoxynucleotidase activity,EC:3.1.3.89



seed.role terms: 3951
seed.reaction terms: 1443
enables_reaction relationships: 2129


## 5. Cleanup (Optional)

In [13]:
# Uncomment to clean up test folder
# import shutil
# if MOCK_CLADE_FOLDER.exists():
#     shutil.rmtree(MOCK_CLADE_FOLDER)
#     print(f"Cleaned up: {MOCK_CLADE_FOLDER}")

## Summary

The `generate_ontology_tables()` function:

1. **Reads** genome features from `db.sqlite` in the clade folder
2. **Maps RAST → seed.role** using RASTSeedMapper with seed.json ontology
3. **Extracts EC** from RAST function strings (e.g., "enzyme (EC 1.1.1.1)")
4. **Extracts** ontology terms (GO, EC, KEGG, COG, PFAM, SO, seed.role)
5. **Enriches** terms with labels and definitions from local parquet files
6. **Extracts** relationships (GO is_a hierarchy, seed.role → enables_reaction → seed.reaction)
7. **Adds EC column** to ontology_terms:
   - KEGG KO → EC from `kegg_ko_ec_mapping.tsv`
   - **GO → EC from `oio:hasDbXref` in statements.parquet** (NEW!)
   - seed.role → EC/TC extracted from label
   - EC terms → identifier copied
8. **Saves** three TSV files to `ontology_data/` subfolder

### Output Files:
| File | Description |
|------|-------------|
| `ontology_terms.tsv` | All terms with: ontology_prefix, identifier, label, definition, **ec** |
| `ontology_definition.tsv` | Ontology prefix definitions |
| `ontology_relationships.tsv` | Term relationships (is_a, enables_reaction) |

### EC Column Sources:
| Ontology | EC Source |
|----------|-----------|
| KEGG | `kegg_ko_ec_mapping.tsv` file |
| GO | `oio:hasDbXref` → `EC:x.x.x.x` in statements.parquet |
| seed.role | Extracted from label: "(EC 1.1.1.1)" |
| EC | Copied from identifier |

### Reference Data Required:
- `seed.json` - RAST → seed.role mapping
- `statements.parquet` - Labels, definitions, relationships, **GO hasDbXref**
- `kegg_ko_definitions.parquet` - KEGG KO definitions
- `cog_definitions.parquet` - COG definitions
- `kegg_ko_ec_mapping.tsv` - KEGG KO → EC mapping